In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import os
import time
import pandas as pd
from src.main_function   import *

In [ ]:
start = time.time()
# Specify setting file 
parameter_file = './setting/parameter1.yaml'
params = load_parameters(parameter_file)
dataset = eval(params['general']['development_dataset'])(data_path=params['path']['data'])
params['features']['mfcc']['win_length'] = int(params['features']['win_length_seconds'] * params['features']['fs'])
params['features']['mfcc']['hop_length'] = int(params['features']['hop_length_seconds'] * params['features']['fs'])
challenge_dataset = eval(params['general']['challenge_dataset'])(data_path=params['path']['data'])
result_path = params['path']['results']
files = []
for item_id, item in enumerate(dataset.train()):
    if item['file'] not in files:
        files.append(item['file'])
files = sorted(files)

In [ ]:
print("Train dataset' feature_extract")
if not os.path.exists("save_data/features"):
    do_feature_extraction(files=files,
                      dataset=dataset,
                      feature_path=params['path']['features'],
                      params=params['features'],
                      overwrite=params['general']['overwrite'])

In [ ]:
print("Train dataset's Feature normalize")
#if not os.path.exists("save_data/features"):
do_feature_normalization(dataset=dataset,
                         feature_normalizer_path=params['path']['feature_normalizers'],
                         feature_path=params['path']['features']
                        )

In [ ]:
print("start trainning by train dataset")
#if not os.path.exists("save_data/features"):
do_system_training(dataset=dataset,
                   model_path=params['path']['models'],
                   feature_normalizer_path=params['path']['feature_normalizers'],
                   feature_path=params['path']['features'],
                   feature_params=params['features'],
                   classifier_params=params['classifier']['parameters'],
                   classifier_method=params['classifier']['method'],
                   clean_audio_errors=params['classifier']['audio_error_handling']['clean_data'],
                   overwrite=params['general']['overwrite']
                   )

In [ ]:
print("create challenge dataset")
challenge_dataset = eval(params['general']['challenge_dataset'])(data_path=params['path']['data'])
result_path = params['path']['results']
files = []
for item_id, item in enumerate(challenge_dataset.test()):
    if item['file'] not in files:
        files.append(item['file'])
files = sorted(files)

In [ ]:
print('test dataset feature extract')
do_feature_extraction(files=files,
                        dataset=challenge_dataset,
                        feature_path=params['path']['features'],
                        params=params['features'],
                        overwrite=params['general']['overwrite'])

In [ ]:
print('classify test dataset')
do_system_testing(dataset=challenge_dataset,
                                    feature_path=params['path']['features'],
                                     result_path=result_path,
                                     model_path=params['path']['models'],
                                    feature_params=params['features'],
                                    classifier_method=params['classifier']['method'],
                                    overwrite=params['general']['overwrite'] or params['general']['challenge_submission_mode']
                                   )

In [ ]:
print("evaluation")
do_system_evaluation(dataset=challenge_dataset,
                     result_path=result_path)
elapsed_time = time.time() - start